In [ ]:
%%capture
!pip install simpletransformers

In [ ]:
# nur mit germeval trainieren
# germeval mit unseren daten trainieren
# nur unsere Daten trainieren

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2022-02-25 16:27:00.148635: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4903074747230010027
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11320098816
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10789051848078428502
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
 xla_global_id: 416903419]

In [ ]:
# import some necessary sub libraries
import pandas as pd
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import f1_score, accuracy_score
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
import os
import tarfile
import re

In [ ]:
def dict_to_df(d):
    df=pd.DataFrame(d.items())
    df.set_index(0, inplace=True)
    return df

In [ ]:
def pack_model(model_path='',file_name=''):
  files = [files for root, dirs, files in os.walk(model_path)][0]
  with tarfile.open(file_name+ '.tar.gz', 'w:gz') as f:
    for file in files:
      f.add(f'{model_path}/{file}')

In [ ]:
def unpack_model(model_name=''):
  tar = tarfile.open(f"{model_name}.tar.gz", "r:gz")
  tar.extractall()
  tar.close()

In [ ]:
def prepare_results(result_dict):
  result_dict["f1_macro"] = result_dict["f1"]["f1_macro"]
  result_dict["f1_micro"] = result_dict["f1"]["f1_micro"]
  result_dict["f1_weighted"] = result_dict["f1"]["f1_weighted"]

  result_dict["precision_macro"] = result_dict["precision"]["precision_macro"]
  result_dict["precision_micro"] = result_dict["precision"]["precision_micro"]
  result_dict["precision_weighted"] = result_dict["precision"]["precision_weighted"]

  result_dict["recall_macro"] = result_dict["recall"]["recall_macro"]
  result_dict["recall_micro"] = result_dict["recall"]["recall_micro"]
  result_dict["recall_weighted"] = result_dict["recall"]["recall_weighted"]

  del result_dict["recall"]
  del result_dict["precision"]
  del result_dict["f1"]

  p_per_class = result_dict["prfs"][0]
  r_per_class = result_dict["prfs"][1]
  f_per_class = result_dict["prfs"][2]
  sample_per_class = result_dict["prfs"][3]

  i = 0
  for number in p_per_class:
    result_dict["precision_class_" + str(i)] = number
    i = i + 1
  
  i = 0
  for number in r_per_class:
    result_dict["recall_class_" + str(i)] = number
    i = i + 1
  
  i = 0
  for number in f_per_class:
    result_dict["f1_score_class_" + str(i)] = number
    i = i + 1
  
  i = 0
  for number in sample_per_class:
    result_dict["sample_class_" + str(i)] = number
    i = i + 1
  
  del result_dict["prfs"]

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, precision_recall_fscore_support, balanced_accuracy_score

def f1_multiclass(labels, preds):
  f1_score_info = {
      "f1_micro": f1_score(labels, preds, average='micro'),
      "f1_macro": f1_score(labels, preds, average='macro'),
      "f1_weighted": f1_score(labels, preds, average='weighted')
  }
  return f1_score_info

def p_multiclass(labels, preds):
  precision_info = {
      "precision_micro": precision_score(labels, preds, average='micro'),
      "precision_macro": precision_score(labels, preds, average='macro'),
      "precision_weighted": precision_score(labels, preds, average='weighted')
  }
  return precision_info

def recall_multiclass(labels, preds):
  recall_info = {
      "recall_micro": recall_score(labels, preds, average='micro'),
      "recall_macro": recall_score(labels, preds, average='macro'),
      "recall_weighted": recall_score(labels, preds, average='weighted')
  }
  return recall_info

In [ ]:
train_test_number = 2
# DONE
#model_name = "bert-base-german-cased"
#model_name = "deepset/gbert-large"
model_name = "deepset/gbert-base"
#model_name = "dbmdz/bert-base-german-cased"
#model_name = "bert-base-german-dbmdz-uncased"
#model_name = "distilbert-base-german-cased"
#model_name = "german-nlp-group/electra-base-german-uncased"
#model_name = "symanto/xlm-roberta-base-snli-mnli-anli-xnli"
#model_name = "bert-base-multilingual-uncased"
#model_name = "xlm-roberta-base"
#model_name = "oliverguhr/german-sentiment-bert"
#model_name = "mdraw/german-news-sentiment-bert"
#model_name = "JP040/bert-german-sentiment-twitter"


model_type = "bert"
label_count = 3
pathTrain = "/content/drive/MyDrive/DH_KFOLD/mitGermeval/Train_Test_"+ str(train_test_number) + "/train.csv"
pathTest = "/content/drive/MyDrive/DH_KFOLD/mitGermeval/Train_Test_"+ str(train_test_number) + "/test.csv"

In [ ]:
train_df = pd.read_csv(pathTrain, encoding="utf-8")
test_df = pd.read_csv(pathTest, encoding="utf-8")
train_df 

,Unnamed: 0,Embedded_text,majority_sentiment
0,0,Unglaublich. Ich hatte @JanJosefLiefers noch...,1
1,1,Kicking of our @EPPGroup bureau meeting in R...,2
2,2,Die Schulbürokratie hält mit der #Corona-Herau...,1
3,4,"""Diese Regierung hat das Vertrauen der Bevölke...",1
4,6,Frankfurter Allgemeine gesamt @FAZ_NET · 7. M...,2
...,...,...,...
22853,28566,Re: Luca Hänni Hi Luca.....heute mal mit der B...,2
22854,28567,Re: Süddeutsche Zeitung Dieses Ergebnis hätte ...,2
22855,28569,RT @Munichfan87: Schnellbremsung der S-Bahn an...,2
22856,28571,@extra3 Vielleicht mal angucken mit wem da gen...,2


In [ ]:
del train_df["Unnamed: 0"]
del test_df["Unnamed: 0"]

In [ ]:
train_df.majority_sentiment.value_counts()

2    15292
1     5957
0     1609
Name: majority_sentiment, dtype: int64

In [ ]:
test_df.majority_sentiment.value_counts()

2    3782
1    1501
0     432
Name: majority_sentiment, dtype: int64

In [ ]:
train_df = train_df[train_df['majority_sentiment'].notna()]

In [ ]:
test_df = test_df[test_df['majority_sentiment'].notna()]

In [ ]:
# nun zum Modell

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "fp16":False,
             "num_train_epochs": 4,
             "overwrite_output_dir":True,
             "train_batch_size": 32, 
             "eval_batch_size": 32 } 

# Create a ClassificationModel
# Attention: Adjust labels accordingly

model = ClassificationModel(
    model_type, model_name,
    num_labels=label_count,
    args=train_args
)

Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
model.train_model(train_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/22858 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/715 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/715 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/715 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/715 [00:00<?, ?it/s]

(2860, 0.30488036282163544)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=accuracy_score, recall=recall_multiclass, precision=p_multiclass, bal_acc=balanced_accuracy_score, f1=f1_multiclass, prfs=precision_recall_fscore_support)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/5715 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/179 [00:00<?, ?it/s]

In [ ]:
prepare_results(result)

In [ ]:
result_df = dict_to_df(result)

In [ ]:
result_df = result_df.sort_values(0)

In [ ]:
result_df

,1
0,
acc,0.820647
bal_acc,0.733504
eval_loss,0.735840
f1_macro,0.747070
f1_micro,0.820647
f1_score_class_0,0.631188
f1_score_class_1,0.737305
f1_score_class_2,0.872718
f1_weighted,0.818895


In [ ]:
# Germeval
# Split 5:
# Split 4: 

In [ ]:
result_df.to_csv("/content/drive/MyDrive/DH_KFOLD/mitGermeval/Evaluation_Germeval/train_test_"+ str(train_test_number) + ".csv", encoding="utf-8")

In [ ]:
pack_model('/content/outputs','/content/drive/MyDrive/DH_KFOLD/mitGermeval/mitGermeval'+"_"+str(train_test_number))